In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from Trainer import Trainer

from WaveValueDataset import WaveValueDataset
from WaveDataset import WaveDataset

from torch.utils.data.dataloader import DataLoader

from Trainer import SequenceTrainer
from WaveModel import PredictionModel, SourcePredictModel, WaveModel
train_length = 256

valuedataset = WaveValueDataset('10k_1orig',train_length+1)
wavedataset = WaveDataset('10k_1orig',train_length)
# M,N,n_embd,n_hidden,n_head,n_layer,dropout=0.2
model = PredictionModel(mic_num=6,tot_timesteps=train_length,n_embd=240,n_hidden=240*3,n_head=8,n_layer=6,dropout=0.1)
wavemodel = WaveModel(mic_num=6,source_num=1,tot_timesteps=train_length,n_embd=240,n_hidden=240*3,n_head=8,n_layer=6,dropout=0.1)

model.load_state_dict(torch.load('SavePretrained.pt'))
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
m=model.to(device)
train=SequenceTrainer(model,valuedataset,batch_size=128,device=device)
trainwave = Trainer(wavemodel,wavedataset,batch_size=128,device=device)
train.to(device)
print(f'Initalized model with {sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6:.2f} M params')
print('device : ', device)

Initialized dataset with 10000 examples
Initialized dataset with 10000 examples
Initalized model with 3.54 M params
device :  cuda:0


In [5]:
torch.save(model.state_dict(),'SavePretrained.pt')

In [ ]:
trainwave.model.train()
trainwave.optimization(epochs=100,eval_interval=3,lr=1e-3)

In [14]:
train.model.train()
train.optimization(epochs=200,eval_interval=5,lr=1e-3,loadingbar=False)

step 0: train loss 0.0988, val loss 0.1145


100%|██████████| 71/71 [00:13<00:00,  5.38it/s]


step 5: train loss 0.0880, val loss 0.1087


100%|██████████| 71/71 [00:13<00:00,  5.38it/s]


step 10: train loss 0.0970, val loss 0.1246


100%|██████████| 71/71 [00:13<00:00,  5.38it/s]


step 15: train loss 0.0851, val loss 0.1118


  6%|▌         | 4/71 [00:00<00:15,  4.27it/s]


KeyboardInterrupt: 

In [16]:
# Use trained model to initialize in source-predict
predictor = SourcePredictModel(train.model.get_trained_transfo(),n_sources=1,n_hidden=32).to(device)
predictor.freeze_body(True)
pred_trainer = Trainer(predictor,wavedataset,batch_size=64,device='cuda:0')
pred_trainer.optimization(9,eval_interval=4,lr=5e-4)




step 0: train loss 0.6269, val loss 0.6124


100%|██████████| 141/141 [00:04<00:00, 32.06it/s]


step 4: train loss 0.0837, val loss 0.0876


100%|██████████| 141/141 [00:04<00:00, 32.02it/s]


step 8: train loss 0.0834, val loss 0.0873


100%|██████████| 141/141 [00:04<00:00, 32.12it/s]


In [18]:
predictor.freeze_body(False)
pred_trainer = Trainer(predictor,wavedataset,batch_size=64,device='cuda:0')
pred_trainer.optimization(20,eval_interval=4,lr=1e-4)

step 0: train loss 0.0342, val loss 0.0367


100%|██████████| 141/141 [00:13<00:00, 10.34it/s]


step 4: train loss 0.0215, val loss 0.0368


100%|██████████| 141/141 [00:13<00:00, 10.35it/s]


step 8: train loss 0.0171, val loss 0.0384


...

100%|██████████| 141/141 [00:13<00:00, 10.33it/s]


step 16: train loss 0.0123, val loss 0.0378


100%|██████████| 141/141 [00:13<00:00, 10.33it/s]


step 19: train loss 0.0103, val loss 0.0381


100%|██████████| 141/141 [00:13<00:00, 10.35it/s]
